In [ ]:
! pip install kaggle
! mkdir /content/Kaggle_API
! mkdir /content/Kaggle_API/.kaggle
from google.colab import files
files.upload()
! ls /content/Kaggle_API -a
! mv /content/kaggle.json /content/Kaggle_API/.kaggle
! ls /content/Kaggle_API/.kaggle -a
! chmod 600 /content/Kaggle_API/.kaggle/kaggle.json
! kaggle datasets download uwrfkaggler/ravdess-emotional-speech-audio
! kaggle datasets download ejlok1/cremad

Saving kaggle.json to kaggle.json
.  ..  .kaggle
.  ..  kaggle.json
Dataset URL: https://www.kaggle.com/datasets/uwrfkaggler/ravdess-emotional-speech-audio
License(s): CC-BY-NC-SA-4.0
100% 428M/429M [00:19<00:00, 24.5MB/s]
100% 429M/429M [00:19<00:00, 22.7MB/s]
Dataset URL: https://www.kaggle.com/datasets/ejlok1/cremad
License(s): ODC Attribution License (ODC-By)
100% 451M/451M [00:24<00:00, 19.4MB/s]
100% 451M/451M [00:24<00:00, 19.3MB/s]


In [ ]:
! unzip ravdess-emotional-speech-audio.zip

In [ ]:
! unzip cremad.zip

In [ ]:
! rm -rf Actor_01 Actor_02 Actor_03 Actor_04 Actor_05 Actor_06 Actor_07 Actor_08 Actor_09 Actor_10 Actor_11 Actor_12 Actor_13 Actor_14
! rm -rf Actor_15 Actor_16 Actor_17 Actor_18 Actor_19 Actor_20 Actor_21 Actor_22 Actor_23 Actor_24

In [ ]:
import pandas as pd
import numpy as np
import os
import librosa
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

Ravdess = "/content/audio_speech_actors_01-24/"
ravdess_directory_list = os.listdir(Ravdess)

file_emotion = []
file_path = []
for dir in ravdess_directory_list:
    actor = os.listdir(Ravdess + dir)
    for file in actor:
        part = file.split('.')[0].split('-')
        file_emotion.append(int(part[2]))
        file_path.append(Ravdess + dir + '/' + file)

emotion_df = pd.DataFrame(file_emotion, columns=['Emotion'])
path_df = pd.DataFrame(file_path, columns=['Path'])
ravdess_df = pd.concat([emotion_df, path_df], axis=1)

ravdess_df.Emotion.replace({1: 'Neutral', 2: 'Calm', 3: 'Happy', 4: 'Sad', 5: 'Angry', 6: 'Fear', 7: 'Disgust', 8: 'Surprise'}, inplace=True)

Crema = "/content/AudioWAV/"
crema_directoryList = os.listdir(Crema)

file_emotion = []
file_path = []

for file in crema_directoryList:
    file_path.append(Crema + file)
    part = file.split('_')

    if part[2] == 'ANG':
        file_emotion.append('Angry')
    elif part[2] == 'DIS':
        file_emotion.append("Disgust")
    elif part[2] == 'FEA':
        file_emotion.append("Fear")
    elif part[2] == 'HAP':
        file_emotion.append('Happy')
    elif part[2] == 'NEU':
        file_emotion.append('Neutral')
    else:
        file_emotion.append('Unknown')

emotion_df = pd.DataFrame(data=file_emotion, columns=['Emotion'])
path_df = pd.DataFrame(data=file_path, columns=['Path'])
crema_df = pd.concat([emotion_df, path_df], axis=1)

final_df = pd.concat([ravdess_df, crema_df], axis=0)
final_df.to_csv("final_df.csv", index=False)

<ipython-input-5-4cbf407b5243>:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ravdess_df.Emotion.replace({1: 'Neutral', 2: 'Calm', 3: 'Happy', 4: 'Sad', 5: 'Angry', 6: 'Fear', 7: 'Disgust', 8: 'Surprise'}, inplace=True)


In [ ]:
def extract_features(data, sampling_rate):
    result = np.array([])
    MFCC = np.mean(librosa.feature.mfcc(y=data, sr=sampling_rate).T, axis=0)
    result = np.hstack((result, MFCC))
    mel_spec = np.mean(librosa.feature.melspectrogram(y=data, sr=sampling_rate).T, axis=0)
    result = np.hstack((result, mel_spec))
    return result


X, Y = [], []
for path, emotion in zip(final_df.Path, final_df.Emotion):
    data, sampling_rate = librosa.load(path, duration=2.5, offset=0.6)
    feature = extract_features(data, sampling_rate)
    X.append(feature)
    Y.append(emotion)


encoder = OneHotEncoder()
Y = encoder.fit_transform(np.array(Y).reshape(-1, 1)).toarray()


X = np.array(X)
x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=0, shuffle=True)


x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout

from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout

model = Sequential()

model.add(Conv1D(256, kernel_size=3, strides=1, padding='same', activation='relu', input_shape=(x_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=3, strides=2, padding='same'))
model.add(Conv1D(128, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3, strides=2, padding='same'))
model.add(Conv1D(64, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3, strides=2, padding='same'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(9, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


model.summary()


history = model.fit(x_train, y_train, batch_size=16, epochs=40, validation_data=(x_test, y_test))

print("Accuracy of the model on the test data:", model.evaluate(x_test, y_test)[1] * 100, '%')


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_35 (Conv1D)                   │ (None, 148, 256)            │           1,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_35 (MaxPooling1D)      │ (None, 74, 256)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_36 (Conv1D)                   │ (None, 74, 128)             │          98,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_36 (MaxPooling1D)      │ (None, 37, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_37 (Conv1D)                   │ (None, 37, 64)              │          24,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_37 (MaxPooling1D)      │ (None, 19, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_11 (Flatten)                 │ (None, 1216)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_31 (Dense)                     │ (None, 64)                  │          77,888 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_22 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_32 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_23 (Dropout)                 │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_33 (Dense)                     │ (None, 9)                   │             297 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 204,361 (798.29 KB)

 Trainable params: 204,361 (798.29 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/40
417/417 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.2052 - loss: 2.1999 - val_accuracy: 0.3030 - val_loss: 1.7260
Epoch 2/40
417/417 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.2787 - loss: 1.7883 - val_accuracy: 0.3246 - val_loss: 1.6143
Epoch 3/40
417/417 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3091 - loss: 1.6994 - val_accuracy: 0.3467 - val_loss: 1.5691
Epoch 4/40
417/417 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3218 - loss: 1.6544 - val_accuracy: 0.3570 - val_loss: 1.5992
Epoch 5/40
417/417 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.3173 - loss: 1.6466 - val_accuracy: 0.3633 - val_loss: 1.5588
Epoch 6/40
417/417 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.3430 - loss: 1.6081 - val_accuracy: 0.3881 - val_loss: 1.5165
Epoch 7/40
417/417 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3479 - loss: 1.5943 - val_accuracy: 0.3647 - val_loss: 1.5211
Epoch 8/40
417/417 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.3547 - loss: 1.5917 - val_accuracy: